# Interfaces: From Protocols to ABCs
Python was already highly successful before ABCs were introduced, and most existing code does not use them at all. <br>
One of the most fundamental interfaces in Python is the sequence protocol. The interpreter goes out of its way to handle objects that provide even a minimal implementation of that protocol, as the next section demonstrates.


## Python Digs Sequences


In [1]:
class Foo:
    def __getitem__(self, pos):
        return range(0, 30, 10)[pos]


In [2]:
f = Foo()

In [3]:
for i in f: print(i)

0
10
20


In [4]:
20 in f

True

In [5]:
15 in f

False

There is no method __iter__ yet Foo instances are iterable because - as a fallback - when Python sees a __getitem__ method, it tries to iterate over the object by calling that method with integer indexes starting with 0. Because Python is smart enough to iterate over Foo instances, it can also make the in operator work even if Foo has no __contains__ method: it does a full scan to check if an item is present. In summary, given the importance of the sequence protocol, in the absence __iter__
and __contains__ Python still manages to make iteration and the in operator work by
invoking __getitem__.

## Monkey-Patching to Implement a Protocol at Runtime
Consider our FrenchDeck again:

In [1]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
 
    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]


This class has a major flaw: it cannot be shuffled.

In [2]:
deck = FrenchDeck()

In [3]:
import traceback
import random

try:
    random.shuffle(deck)
except:
    traceback.print_exc()
    

Traceback (most recent call last):
  File "<ipython-input-3-47e028c7bd17>", line 5, in <module>
    random.shuffle(deck)
  File "C:\Anaconda2\envs\py36\lib\random.py", line 277, in shuffle
    x[i], x[j] = x[j], x[i]
TypeError: 'FrenchDeck' object does not support item assignment


The problem is that shuffle operates by swapping items inside the collection, and FrenchDeck only implements the immutable sequence protocol. Mutable sequences must also provide a __setitem__ method. We can monkey-patch FrenchDeck to make it mutable.

In [6]:
def set_card(deck, position, card):
    deck._cards[position] = card

FrenchDeck.__setitem__ = set_card

In [8]:
random.shuffle(deck) # no errors here...

See the [docs for Python's data model](https://docs.python.org/3/reference/datamodel.html)...

[object.__getitem__(self, key)](https://docs.python.org/3/reference/datamodel.html#object.__getitem__): Called to implement evaluation of self[key]. For sequence types, the accepted keys should be integers and slice objects. <br>
<br>
[object.__setitem__(self, key, value)](https://docs.python.org/3/reference/datamodel.html#object.__setitem__): Called to implement assignment to self[key]. Same note as for __getitem__(). This should only be implemented for mappings if the objects support changes to the values for keys, or if new keys can be added, or for sequences if elements can be replaced. 

In [10]:
help(slice)

Help on class slice in module builtins:

class slice(object)
 |  slice(stop)
 |  slice(start, stop[, step])
 |  
 |  Create a slice object.  This is used for extended slicing (e.g. a[0:10:2]).
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __le__(self, value, /)
 |      Return self<=value.
 |  
 |  __lt__(self, value, /)
 |      Return self<value.
 |  
 |  __ne__(self, value, /)
 |      Return self!=value.
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.
 |  
 |  __reduce__(...)
 |      Return state information for pickling.
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  indices(...)
 |      S.indices(len) -> (start, stop, stride)
 |      
 | 

In [12]:
s = "ABCDEFGHIJKL"
sl = slice(0,4)
print(s[sl])

ABCD


Square brackets following a sequence denote either indexing or slicing depending on what's inside the brackets:

In [15]:
"Python rocks"[1]    # index

'y'

In [16]:
"Python rocks"[1:10:2]    # slice

'yhnrc'

In [17]:
def __getitem__(self, index):
    if isinstance(index, int):
        ...    # process index as an integer
    elif isinstance(index, slice):
        start, stop, step = index.indices(len(self))    # index is a slice
        ...    # process slice
    else:
        raise TypeError("index must be int or slice")

In [18]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
 
    def __len__(self):
        return len(self._cards)

    def __getitem__(self, index):
        print(type(index))
        return self._cards[index]


In [19]:
deck = FrenchDeck()

In [20]:
deck[1]

<class 'int'>


Card(rank='3', suit='spades')

In [22]:
deck[slice(1, 10, 2)]

<class 'slice'>


[Card(rank='3', suit='spades'),
 Card(rank='5', suit='spades'),
 Card(rank='7', suit='spades'),
 Card(rank='9', suit='spades'),
 Card(rank='J', suit='spades')]

In [25]:
import traceback

try:
    deck[1, 10, 2]
except:
    traceback.print_exc()

<class 'tuple'>


Traceback (most recent call last):
  File "<ipython-input-25-529771141a3c>", line 4, in <module>
    deck[1, 10, 2]
  File "<ipython-input-18-a129899cfb03>", line 17, in __getitem__
    return self._cards[index]
TypeError: list indices must be integers or slices, not tuple


## Subclassing an ABC
Following Martelli’s advice, we’ll leverage an existing ABC, collections.MutableSequence, before daring to invent our own.

In [28]:
import collections

help(collections.MutableSequence)

Help on class MutableSequence in module collections.abc:

class MutableSequence(Sequence)
 |  All the operations on a read-only sequence.
 |  
 |  Concrete subclasses must override __new__ or __init__,
 |  __getitem__, and __len__.
 |  
 |  Method resolution order:
 |      MutableSequence
 |      Sequence
 |      Reversible
 |      Collection
 |      Sized
 |      Iterable
 |      Container
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __delitem__(self, index)
 |  
 |  __iadd__(self, values)
 |  
 |  __setitem__(self, index, value)
 |  
 |  append(self, value)
 |      S.append(value) -- append value to the end of the sequence
 |  
 |  clear(self)
 |      S.clear() -> None -- remove all items from S
 |  
 |  extend(self, values)
 |      S.extend(iterable) -- extend sequence by appending elements from the iterable
 |  
 |  insert(self, index, value)
 |      S.insert(index, value) -- insert value before index
 |  
 |  pop(self, index=-1)
 |      S.pop([index]) -> item --

In [29]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(collections.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

    def __setitem__(self, position, value):
        self._cards[position] = value

    def __delitem__(self, position):
        del self._cards[position]

    def insert(self, position, value):
        self._cards.insert(position, value)


We are required to implement __delitem__ and insert as these are abstract methods of MutableSequence.

## The Numbers Tower of ABCs